In [14]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [15]:
 #Charger les données
df = pd.read_csv('/home/user/Desktop/PFA/real_time_pipeline_project/data/ML/df_model.csv')

In [16]:
df

,quartier,nombre_vehicules,niveau_trafic,temperature,precipitations,vent,humidite,densite_population,taux_chomage,niveau_vie,etat,timestamp
0,Hay Hassani,774,Faible,14.46,2.60,31.69,53.00,10467.13,17.08,Bas,Pas de vol,2025-01-16 05:58:27
1,Hay Hassani,939,Moyen,13.71,2.60,25.92,53.00,10467.13,17.08,Bas,Pas de vol,2025-01-12 21:00:27
2,Hay Hassani,456,Faible,20.57,0.80,28.67,44.00,10467.13,17.08,Bas,Pas de vol,2025-01-15 09:10:27
3,Hay Hassani,1225,Élevé,27.17,0.64,23.54,43.20,10467.13,17.08,Bas,Pas de vol,2025-01-14 09:26:27
4,Hay Hassani,377,Faible,14.98,2.44,24.93,52.20,10467.13,17.08,Bas,Pas de vol,2025-01-12 13:59:27
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Gauthier,1003,Faible,13.32,1.36,33.36,46.80,11768.83,8.52,Élevé,Résolu,2025-01-12 14:11:27
49996,Gauthier,1170,Moyen,28.53,1.57,9.07,47.85,11768.83,8.52,Élevé,Résolu,2025-01-13 14:36:27
49997,Gauthier,219,Moyen,25.67,2.24,35.18,51.20,11768.83,8.52,Élevé,Résolu,2025-01-15 20:11:27
49998,Gauthier,77,Très Élevé,15.12,0.71,15.40,43.55,11768.83,8.52,Élevé,Résolu,2025-01-17 18:32:27


In [17]:
# Prétraitement général
df['niveau_vie'] = df['niveau_vie'].map({'Bas': 0, 'Moyen': 1, 'Élevé': 2})
df['niveau_trafic'] = df['niveau_trafic'].map({'Faible': 0, 'Moyen': 1, 'Élevé': 2, 'Très Élevé': 3})
df['heure'] = pd.to_datetime(df['timestamp']).dt.hour

In [18]:
# Séparer les données par quartier
quartiers = ['Hay Hassani', 'Sidi Bernoussi', 'Maarif', 'Ain Diab', 'Gauthier']


In [19]:
df.columns

Index(['quartier', 'nombre_vehicules', 'niveau_trafic', 'temperature',
       'precipitations', 'vent', 'humidite', 'densite_population',
       'taux_chomage', 'niveau_vie', 'etat', 'timestamp', 'heure'],
      dtype='object')

In [20]:
df.etat.value_counts()

etat
Pas de vol    25000
En cours      12500
Résolu        12500
Name: count, dtype: int64

In [21]:
# Encodage manuel de la colonne 'etat'
etat_mapping = {
    'Pas de vol': 0,
    'En cours': 1,
    'Résolu': 1  # En cours et Résolu sont regroupés sous le même label
}

# Appliquer l'encodage
df['etat'] = df['etat'].map(etat_mapping)

In [22]:
df.etat

0        0
1        0
2        0
3        0
4        0
        ..
49995    1
49996    1
49997    1
49998    1
49999    1
Name: etat, Length: 50000, dtype: int64

In [23]:
# 3. Prétraitement des données
quartiers = df['quartier'].unique()
models = {}

# 4. Boucle pour entraîner un modèle pour chaque quartier
for quartier in quartiers:
    print(f"🔄 Entraînement du modèle pour {quartier}...")

    # Filtrer les données pour le quartier actuel
    quartier_data = df[df['quartier'] == quartier]

    # Préparer les caractéristiques et la cible
    X = quartier_data[['nombre_vehicules', 'niveau_trafic', 'temperature',
                       'precipitations', 'vent', 'humidite', 'densite_population',
                       'taux_chomage', 'niveau_vie', 'heure']]
    y = quartier_data['etat']  # État de l'incident (target)

    # Diviser les données en ensemble d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 5. Entraîner un modèle Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 6. Évaluation du modèle
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"🎯 Précision pour {quartier} : {accuracy:.2f}")
  

    # 7. Sauvegarder le modèle
    model_path = f'/home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_{quartier}.pkl'
    joblib.dump(model, model_path)
    print(f"💾 Modèle sauvegardé pour {quartier} : {model_path}")

    

🔄 Entraînement du modèle pour Hay Hassani...
🎯 Précision pour Hay Hassani : 0.51
💾 Modèle sauvegardé pour Hay Hassani : /home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_Hay Hassani.pkl
🔄 Entraînement du modèle pour Sidi Bernoussi...
🎯 Précision pour Sidi Bernoussi : 0.48
💾 Modèle sauvegardé pour Sidi Bernoussi : /home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_Sidi Bernoussi.pkl
🔄 Entraînement du modèle pour Maarif...
🎯 Précision pour Maarif : 0.51
💾 Modèle sauvegardé pour Maarif : /home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_Maarif.pkl
🔄 Entraînement du modèle pour Ain Diab...
🎯 Précision pour Ain Diab : 0.49
💾 Modèle sauvegardé pour Ain Diab : /home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_Ain Diab.pkl
🔄 Entraînement du modèle pour Gauthier...
🎯 Précision pour Gauthier : 0.50
💾 Modèle sauvegardé pour Gauthier : /home/user/Desktop/PFA/real_time_pipeline_project

In [24]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# import joblib

# # Define a dictionary of models to test
# available_models = {
#     'Random Forest': RandomForestClassifier(),
#     'Logistic Regression': LogisticRegression(max_iter=1000),
#     'Support Vector Machine': SVC(),
#     'Decision Tree': DecisionTreeClassifier()
# }

# # Directory to save the models
# base_path = '/home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/'

# # Train multiple models for each neighborhood and select the best one
# best_models = {}

# for quartier in quartiers:
#     print(f"🔄 Entraînement pour le quartier {quartier}...")
    
#     # Filtrer les données pour le quartier actuel
#     quartier_data = df[df['quartier'] == quartier]

#     X = quartier_data[['nombre_vehicules', 'niveau_trafic', 'temperature',
#                        'precipitations', 'vent', 'humidite', 'densite_population',
#                        'taux_chomage', 'niveau_vie', 'heure']]
    
#     y = quartier_data['etat']
    
#     # Diviser en ensembles d'entraînement et de test
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     best_accuracy = 0
#     best_model = None
    
#     for model_name, model in available_models.items():
#         # Entraîner le modèle
#         model.fit(X_train, y_train)
#         predictions = model.predict(X_test)
#         accuracy = accuracy_score(y_test, predictions)
#         print(f"Accuracy pour {model_name}: {accuracy:.2f}")
        
#         # Sélectionner le meilleur modèle
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_model = (model_name, model)
    
#     # Construire le chemin complet du fichier
#     model_path = f'/home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/model_{quartier}.pkl'
    
#     # Sauvegarder le meilleur modèle au format .pkl
#     joblib.dump(best_model[1], model_path)
#     print(f"📁 Modèle sauvegardé: {model_path}")
    
#     best_models[quartier] = {
#         'model_name': best_model[0],
#         'accuracy': best_accuracy,
#         'model': best_model[1],
#         'file': model_path
#     }
#     print(f"✅ Meilleur modèle pour {quartier}: {best_model[0]} avec une accuracy de {best_accuracy:.2f}")


In [25]:
# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Directory to save the models
# base_path = '/home/user/Desktop/PFA/real_time_pipeline_project/machine_learning/model/'

# # Fonction pour créer différents modèles
# def create_model_1(input_shape):
#     model = Sequential([
#         Dense(128, activation='relu', input_shape=(input_shape,)),
#         BatchNormalization(),
#         Dropout(0.3),
#         Dense(64, activation='relu'),
#         Dropout(0.3),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model

# def create_model_2(input_shape):
#     model = Sequential([
#         Dense(256, activation='relu', input_shape=(input_shape,)),
#         BatchNormalization(),
#         Dropout(0.4),
#         Dense(128, activation='relu'),
#         Dropout(0.4),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model

# # Dictionnaire de modèles disponibles
# available_models = {
#     'Model_1': create_model_1,
#     'Model_2': create_model_2,
# }

# # Entraîner plusieurs modèles pour chaque quartier
# best_models = {}

# for quartier in quartiers:
#     print(f"🔄 Entraînement pour le quartier {quartier}...")
    
#     # Filtrer les données pour le quartier actuel
#     quartier_data = df[df['quartier'] == quartier]
#     X = quartier_data[['nombre_vehicules', 'niveau_trafic', 'temperature',
#                        'precipitations', 'vent', 'humidite', 'densite_population',
#                        'taux_chomage', 'niveau_vie', 'heure']].values
#     y = quartier_data['etat'].values

#     # Encodage des labels si nécessaire
#     label_encoder = LabelEncoder()
#     y = label_encoder.fit_transform(y)

#     # Diviser en ensembles d'entraînement et de validation
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

#     input_shape = X_train.shape[1]
#     best_accuracy = 0
#     best_model_name = None
#     best_model = None

#     for model_name, model_function in available_models.items():
#         print(f"⏳ Entraînement avec {model_name}...")
        
#         # Créer le modèle
#         model = model_function(input_shape)

#         # Chemin pour sauvegarder le modèle pendant l'entraînement
#         model_path = f"{base_path}model_{quartier.replace(' ', '_')}_{model_name}.h5"
#         checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

#         # Entraîner le modèle
#         history = model.fit(
#             X_train, y_train,
#             validation_data=(X_val, y_val),
#             epochs=50,
#             batch_size=32,
#             callbacks=[checkpoint],
#             verbose=2
#         )

#         # Récupérer la meilleure précision sur les données de validation
#         val_accuracy = max(history.history['val_accuracy'])
#         print(f"Validation accuracy pour {model_name}: {val_accuracy:.2f}")
        
#         # Sélectionner le meilleur modèle
#         if val_accuracy > best_accuracy:
#             best_accuracy = val_accuracy
#             best_model_name = model_name
#             best_model = model

#     # Sauvegarder le meilleur modèle pour le quartier
#     final_model_path = f"{base_path}model_{quartier.replace(' ', '_')}_best.h5"
#     best_model.save(final_model_path)
#     print(f"📁 Meilleur modèle sauvegardé : {final_model_path}")

#     best_models[quartier] = {
#         'model_name': best_model_name,
#         'accuracy': best_accuracy,
#         'model_file': final_model_path
#     }
#     print(f"✅ Meilleur modèle pour {quartier}: {best_model_name} avec une accuracy de {best_accuracy:.2f}")


In [26]:
pip install tensorflow

84.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
